In [1]:
import datetime as dt
from harmony import BBox, Client, Collection, Request, LinkType
import s3fs
import xarray as xr
import json
import requests

In [2]:
# Initialize Harmony client
harmony_client = Client()

In [3]:
# Define dataset and parameters
dataset_short_name = 'MUR-JPL-L4-GLOB-v4.1'
spatial_bbox = BBox(-125.469, 15.820, -99.453, 35.859)  # Define your bounding box coordinates
temporal_start = dt.datetime(2021, 3, 10, 1)
temporal_stop = dt.datetime(2021, 3, 10, 2)

In [4]:
# Create the request
request = Request(
    collection=Collection(id=dataset_short_name),
    spatial=spatial_bbox,
    temporal={
        'start': temporal_start,
        'stop': temporal_stop
    }
)

In [5]:
# Submit the request and handle potential errors
try:
    job_id = harmony_client.submit(request)
    harmony_client.wait_for_processing(job_id)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    exit(1)
except json.JSONDecodeError as e:
    print(f"JSON decode error: {e}")
    exit(1)

Request failed: Expecting value: line 1 column 1 (char 0)


In [6]:
# Retrieve the result URLs
try:
    results = harmony_client.result_urls(job_id, link_type=LinkType.s3)
    urls = list(results)
except Exception as e:
    print(f"Failed to retrieve result URLs: {e}")
    exit(1)

Failed to retrieve result URLs: name 'job_id' is not defined


In [7]:
# Retrieve AWS credentials
try:
    creds = harmony_client.aws_credentials()
except Exception as e:
    print(f"Failed to retrieve AWS credentials: {e}")
    exit(1)

Failed to retrieve AWS credentials: Expecting value: line 1 column 1 (char 0)


In [8]:
# Set up S3 file system
try:
    s3_fs = s3fs.S3FileSystem(
        key=creds['aws_access_key_id'],
        secret=creds['aws_secret_access_key'],
        token=creds['aws_session_token'],
        client_kwargs={'region_name': 'us-west-2'}
    )
except Exception as e:
    print(f"Failed to set up S3 file system: {e}")
    exit(1)

Failed to set up S3 file system: name 'creds' is not defined


In [9]:
# Open and read the subsetted file
try:
    with s3_fs.open(urls[0], mode='rb') as f:
        ds = xr.open_dataset(f)
    # Plot the data
    ds.analysed_sst.plot()
except Exception as e:
    print(f"Failed to open and read the subsetted file: {e}")
    exit(1)

Failed to open and read the subsetted file: name 's3_fs' is not defined
